In [ ]:
!pip install -q mlxtend openpyxl

In [ ]:
# Tắt cảnh báo DeprecationWarning từ jupyter_client (cho đỡ spam)
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [ ]:
# DP1 — Imports & cấu hình I/O (IN_DIR / OUT_DIR như yêu cầu)

import os, re, unicodedata
import numpy as np
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules
from collections import Counter

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
# Thư mục IN/OUT
IN_DIR  = "/content/drive/MyDrive/DACNTT/Data"
OUT_DIR = "/content/drive/MyDrive/DACNTT/DataAfterProcessing"
os.makedirs(OUT_DIR, exist_ok=True)

# Đường dẫn input
PATH_PRODUCTS   = os.path.join(IN_DIR, "products_data.xlsx")
PATH_BRANDS     = os.path.join(IN_DIR, "brands_data.xlsx")
PATH_CATEGORIES = os.path.join(IN_DIR, "categories_data.xlsx")
PATH_COUNTRIES  = os.path.join(IN_DIR, "countries_of_origin.csv")
PATH_BILLS      = os.path.join(IN_DIR, "all_barcodes.xlsx")

# Đường dẫn output (4 file cuối)
PRODUCTS_FINAL_PATH    = os.path.join(OUT_DIR, "products_final.csv")
TRANSACTIONS_LONG_PATH = os.path.join(OUT_DIR, "transactions_long.csv")
ITEM_RULES_PATH        = os.path.join(OUT_DIR, "item_rules.csv")
CATEGORY_RULES_PATH    = os.path.join(OUT_DIR, "category_rules.csv")

In [ ]:
# Tham số khai khoáng luật
MIN_SUPPORT_ITEM       = 0.003
MIN_CONF_ITEM          = 0.15
MIN_LIFT_ITEM          = 1.05

MIN_SUPPORT_CATEGORY   = 0.003
MIN_CONF_CATEGORY      = 0.15
MIN_LIFT_CATEGORY      = 1.02

# Các hàm Utilities

In [ ]:
def ensure_str(x):
    return "" if pd.isna(x) else str(x)

def normalize_unicode(s):
    """Chuẩn hoá Unicode (NFKC) và khoảng trắng."""
    s = ensure_str(s).strip()
    s = unicodedata.normalize("NFKC", s)
    s = re.sub(r"\\s+", " ", s)
    return s

def standardize_content(s):
    """
    CHUẨN HOÁ NỘI DUNG (GIỮ DẤU):
    Chỉ chuẩn hoá Unicode và lowercase.
    Dùng cho product_name, description, và tags.
    """
    s = normalize_unicode(s).lower()
    return s

def standardize_for_mapping(s):
    """
    CHUẨN HOÁ ĐỂ MAPPING (BỎ DẤU):
    lowercase + bỏ dấu tiếng Việt để so khớp/mapping.
    Dùng cho brand, category, country khi tra cứu.
    """
    s = normalize_unicode(s).lower()
    nfkd = unicodedata.normalize("NFKD", s)
    s = "".join([c for c in nfkd if not unicodedata.combining(c)])
    return s

def standardize_id(x):
    # ID/barcode dạng uppercase, bỏ khoảng trắng
    return ensure_str(x).strip().upper()

def detect_col(df, candidates):
    cols = {c.lower(): c for c in df.columns}
    for name in candidates:
        if name.lower() in cols:
            return cols[name.lower()]
    return None

def tokenize_basic(s):
    """
    Tokenize từ nội dung (đã được giữ dấu).
    """
    # Dùng hàm standardize_content (giữ dấu)
    s_norm = standardize_content(s)

    # Regex này được cập nhật để chấp nhận ký tự tiếng Việt có dấu
    pattern = r"[a-z0-9àáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ]+"

    return [t for t in re.findall(pattern, s_norm) if len(t) >= 2]

def price_bucketize(series):
    s = pd.to_numeric(series, errors="coerce")
    if s.notna().sum() > 10:
        q1, q2 = s.quantile([0.33, 0.66]).values
    else:
        q1 = q2 = s.median()
    def bucket(x):
        if pd.isna(x): return "price_unk"
        if x <= q1:  return "price_cheap"
        if x <= q2:  return "price_mid"
        return "price_premium"
    return s.map(bucket)

def build_map(df,
              raw_cols=("raw","source","old","brand_raw","category_raw","xuat_xu","country_raw"),
              can_cols=("canonical","target","new","brand","category","country","ten")):
    """
    Tạo dict: chuẩn_hoá_mapping(raw) -> canonical
    (Hàm này dùng standardize_for_mapping để tạo key)
    """
    if df is None or df.empty:
        return None
    cols = {c.lower(): c for c in df.columns}

    raw_col = None
    for name in raw_cols:
        if name.lower() in cols:
            raw_col = cols[name.lower()]
            break

    can_col = None
    for name in can_cols:
        if name.lower() in cols:
            can_col = cols[name.lower()]
            break

    if not raw_col or not can_col:
        if len(df.columns) >= 2:
            raw_col, can_col = df.columns[0], df.columns[1]
        else:
            return None

    m = {}
    for _, r in df.iterrows():
        # Key của map là dạng BỎ DẤU
        k = standardize_for_mapping(r[raw_col])
        # Giá trị (canonical) là dạng CÓ DẤU (chuẩn hoá Unicode)
        v = normalize_unicode(r[can_col])
        if k:
            m[k] = v
    return m

def apply_mapping(val, map_dict, default_on_empty=None):
    """
    map_dict: dict [đã bỏ dấu] -> [canonical có dấu]
    """
    # Giá trị input (val) cũng được BỎ DẤU để tra cứu
    v_norm = standardize_for_mapping(val)

    if map_dict is None:
        # Nếu không có map, trả về giá trị đã chuẩn hoá unicode (có dấu)
        mapped_val = normalize_unicode(val)
        return default_on_empty if (default_on_empty is not None and v_norm == "") else mapped_val

    mapped = map_dict.get(v_norm, None)
    if mapped is None:
        # Nếu tra cứu thất bại, trả về giá trị đã chuẩn hoá unicode (có dấu)
        mapped_val = normalize_unicode(val)
        return default_on_empty if (default_on_empty is not None and v_norm == "") else mapped_val

    # Trả về giá trị canonical (có dấu) từ map
    return mapped

# Load raw data & mapping

In [ ]:
# Load dữ liệu thô & mapping

def read_required(path, kind="file"):
    if not path or not os.path.exists(path):
        raise RuntimeError(f"Không tìm thấy {kind} tại '{path}'. Kiểm tra IN_DIR và tên file.")
    try:
        if path.lower().endswith(".csv"):
            return pd.read_csv(path)
        else:
            # Excel: mặc định engine openpyxl
            return pd.read_excel(path, engine="openpyxl")
    except Exception as e:
        raise RuntimeError(f"Lỗi khi đọc {kind} từ '{path}': {e}")

def read_optional(path):
    if not path or not os.path.exists(path):
        return pd.DataFrame()
    try:
        if path.lower().endswith(".csv"):
            return pd.read_csv(path)
        else:
            return pd.read_excel(path, engine="openpyxl")
    except Exception:
        return pd.DataFrame()

# Đọc chính
products_raw = read_required(PATH_PRODUCTS, kind="products_data")
bills_df     = read_required(PATH_BILLS, kind="all_barcodes")

# Đọc mapping (tuỳ chọn)
brands_raw   = read_optional(PATH_BRANDS)
cats_raw     = read_optional(PATH_CATEGORIES)
countries_raw= read_optional(PATH_COUNTRIES)

brand_map   = build_map(brands_raw)
cat_map     = build_map(cats_raw)
country_map = build_map(countries_raw)

# Cleaning products

In [ ]:
def clean_products(products_df, brand_map=None, cat_map=None, country_map=None):
    if products_df is None or products_df.empty:
        raise RuntimeError("products_data is missing or empty.")

    # Phát hiện cột
    col_id   = detect_col(products_df, ["product_id","id","sku","barcode","ma_vach"])
    col_name = detect_col(products_df, ["product_name","name","title","ten_san_pham"])
    col_desc = detect_col(products_df, ["description","desc","mo_ta","mota"])
    col_brand= detect_col(products_df, ["brand","thuong_hieu"])
    col_cat  = detect_col(products_df, ["category","loai","nganh_hang","loai_san_pham"])
    col_org  = detect_col(products_df, ["country_of_origin","country","origin","xuat_xu"])
    col_price= detect_col(products_df, ["price","gia","unit_price","unitprice","gia_ban"])

    out = products_df.copy()

    # Chuẩn hóa tên cột
    if col_id:   out.rename(columns={col_id:"product_id"}, inplace=True)
    else:        out["product_id"] = ""
    if col_name: out.rename(columns={col_name:"product_name"}, inplace=True)
    else:        out["product_name"] = ""
    if col_desc and col_desc!="description": out.rename(columns={col_desc:"description"}, inplace=True)
    elif "description" not in out.columns:  out["description"] = ""
    if col_brand and col_brand!="brand": out.rename(columns={col_brand:"brand"}, inplace=True)
    elif "brand" not in out.columns:    out["brand"] = ""
    if col_cat and col_cat!="category": out.rename(columns={col_cat:"category"}, inplace=True)
    elif "category" not in out.columns: out["category"] = ""
    if col_org and col_org!="country_of_origin": out.rename(columns={col_org:"country_of_origin"}, inplace=True)
    elif "country_of_origin" not in out.columns: out["country_of_origin"] = ""
    if col_price and col_price!="price": out.rename(columns={col_price:"price"}, inplace=True)
    elif "price" not in out.columns:    out["price"] = np.nan

    # 1. Chuẩn hoá ID
    out["product_id"] = out["product_id"].map(standardize_id)

    # 2. Chuẩn hoá NỘI DUNG (GIỮ DẤU)
    out["product_name"] = out["product_name"].map(standardize_content)
    out["description"] = out["description"].map(standardize_content)

    # 3. Chuẩn hoá và MAPPING (dùng logic bỏ dấu để tra cứu, trả về giá trị có dấu)

    # Lưu lại giá trị thô (đã chuẩn hoá unicode có dấu)
    out["brand_raw"] = out["brand"].map(normalize_unicode)
    out["category_raw"] = out["category"].map(normalize_unicode)
    out["country_raw"] = out["country_of_origin"].map(normalize_unicode)

    # Áp dụng mapping
    out["brand"] = out["brand_raw"].map(lambda v: apply_mapping(v, brand_map, default_on_empty="UNKNOWN_BRAND"))
    out["category"] = out["category_raw"].map(lambda v: apply_mapping(v, cat_map, default_on_empty="UNCATEGORIZED"))
    out["country_of_origin"] = out["country_raw"].map(lambda v: apply_mapping(v, country_map, default_on_empty="UNK"))

    # ============================

    # Giá & bucket
    out["price"] = pd.to_numeric(out["price"], errors="coerce")
    out["price_bucket"] = price_bucketize(out["price"])

    # Tags (Hàm make_tags này giờ sẽ dùng tokenize_basic đã giữ dấu)
    def make_tags(row):
        toks = []
        # Các token này giờ sẽ CÓ DẤU
        toks += tokenize_basic(row.get("product_name","") + " " + row.get("description",""))

        # Các tag này là tag canonical CÓ DẤU
        for c in ["brand","category","country_of_origin","price_bucket"]:
            v = ensure_str(row.get(c, ""))
            if v: toks.append(v.lower().replace(" ", "_")) # vd: "dầu_gội,_dầu_xả"

        seen, ret = set(), []
        for t in toks:
            if t not in seen:
                seen.add(t); ret.append(t)
        return ret

    out["tags"] = out.apply(make_tags, axis=1)
    out["tags_str"] = out["tags"].map(lambda xs: " ".join(xs))

    # Lưu file cuối
    out.to_csv(PRODUCTS_FINAL_PATH, index=False, encoding="utf-8")
    return out

# Chạy lại quá trình clean
products = clean_products(products_raw, brand_map, cat_map, country_map)

print("Đã xử lý xong! Dữ liệu product_name, description và tags giờ đã giữ lại dấu:")
products.head(5)


Đã xử lý xong! Dữ liệu product_name, description và tags giờ đã giữ lại dấu:


,_id,product_id,product_name,price,description,brand,country_of_origin,category,brand_raw,category_raw,country_raw,price_bucket,tags,tags_str
0,68311cba419bc51ab2ee5aa3,8934868172239,dầu gội nam sạch gàu clear men hương gió biển ...,249000,"ưu điểm nổi bật:– sạch gàu, hết ngứa:công nghệ...",Clear Men,UNK,"Dầu Gội, Dầu Xả",68311cba419bc51ab2ee5aa2,682fdf11419bc50c9f24af11,,price_premium,"[dầu, gội, nam, sạch, gàu, clear, men, hương, ...",dầu gội nam sạch gàu clear men hương gió biển ...
1,68311cba419bc51ab2ee5aa5,8934868152576,dầu gội sạch gàu clear bạc hà mát lạnh chai 1.4kg,370000,thời tiết nóng ẩm cùng khói bụi là nguyên nhân...,Clear,UNK,"Dầu Gội, Dầu Xả",68311cba419bc51ab2ee5aa4,682fdf11419bc50c9f24af11,,price_premium,"[dầu, gội, sạch, gàu, clear, bạc, hà, mát, lạn...",dầu gội sạch gàu clear bạc hà mát lạnh chai 4k...
2,68311cbb419bc51ab2ee5aa6,8934868172253,dầu gội nam sạch gàu clear men hương phong lữ ...,249000,đặc điểm sản phẩm:dầu gội sạch gàu clear men p...,Clear Men,UNK,"Dầu Gội, Dầu Xả",68311cba419bc51ab2ee5aa2,682fdf11419bc50c9f24af11,,price_premium,"[dầu, gội, nam, sạch, gàu, clear, men, hương, ...",dầu gội nam sạch gàu clear men hương phong lữ ...
3,68311cbb419bc51ab2ee5aa7,8934868168584,dầu gội nam sạch gàu clear men bạc hà dây 10 g...,15000,dầu gội clear men mát lạnh bạc hà chứa hoạt ch...,Clear Men,UNK,"Dầu Gội, Dầu Xả",68311cba419bc51ab2ee5aa2,682fdf11419bc50c9f24af11,,price_cheap,"[dầu, gội, nam, sạch, gàu, clear, men, bạc, hà...",dầu gội nam sạch gàu clear men bạc hà dây 10 g...
4,68311cbe419bc51ab2ee5aa8,8934868152811,dầu gội nam sạch sâu trị gàu ngứa vi khuẩn cle...,249000,dầu gội nam clear men sạch sâu sạch gàu ngừa v...,Clear Men,UNK,"Dầu Gội, Dầu Xả",68311cba419bc51ab2ee5aa2,682fdf11419bc50c9f24af11,,price_premium,"[dầu, gội, nam, sạch, sâu, trị, gàu, ngứa, vi,...",dầu gội nam sạch sâu trị gàu ngứa vi khuẩn cle...


# Tách bill thành dạng long (mỗi row cho 1 sản phẩm)

In [ ]:
# Bills -> dạng long (bill_id, product_id) -> transactions_long.csv

def bills_to_long(bills_df, products_df):
    if bills_df is None or bills_df.empty:
        return pd.DataFrame(columns=["bill_id","product_id"])

    bill_id_col = None
    prod_id_col, prod_ids_col = None, None
    for c in bills_df.columns:
        cl = c.strip().lower()
        if cl in ["bill_id","id_bill","order_id","ma_hoa_don","so_hd"]:
            bill_id_col = c
        elif cl in ["product_id","sku","barcode","productcode","ma_vach","ma_san_pham"]:
            prod_id_col = c
        elif cl in ["product_ids","product_list","items","product_codes","ds_barcode"]:
            prod_ids_col = c

    df = bills_df.copy()
    if bill_id_col is None:
        df["__bill__"] = np.arange(1, len(df)+1).astype(str)
        bill_id_col = "__bill__"

    # Trường hợp 1 bill có list sản phẩm trong 1 ô
    if prod_ids_col and prod_ids_col in df.columns:
        rows = []
        for _, r in df.iterrows():
            bid = ensure_str(r[bill_id_col])
            raw = ensure_str(r[prod_ids_col])
            items = re.split(r"[,\s;]+", raw.strip())
            items = [standardize_id(x) for x in items if x.strip()]
            rows.extend([(bid, pid) for pid in items])
        long_df = pd.DataFrame(rows, columns=["bill_id","product_id"])

    # Trường hợp mỗi dòng là 1 (bill, product_id)
    elif prod_id_col and prod_id_col in df.columns:
        long_df = df[[bill_id_col, prod_id_col]].copy()
        long_df.columns = ["bill_id","product_id"]
        long_df["bill_id"] = long_df["bill_id"].map(ensure_str)
        long_df["product_id"] = long_df["product_id"].map(standardize_id)

    else:
        long_df = pd.DataFrame(columns=["bill_id","product_id"])

    # Chỉ giữ product_id tồn tại trong products
    valid_ids = set(products_df["product_id"].tolist())
    long_df = long_df[long_df["product_id"].isin(valid_ids)].copy()

    long_df.to_csv(TRANSACTIONS_LONG_PATH, index=False, encoding="utf-8")
    return long_df

bills_long = bills_to_long(bills_df, products)

In [ ]:
# Luật theo HÀNG (item co-purchase) -> item_rules.csv
# - Sinh luật cặp (A -> B) bằng đếm cặp xuất hiện trong bill
# - Lưu cả ID lẫn tên sản phẩm cho dễ đọc

from collections import Counter

def mine_item_rules_pairwise(bills_long,
                             products_df,
                             min_support=MIN_SUPPORT_ITEM,
                             min_conf=MIN_CONF_ITEM,
                             min_lift=MIN_LIFT_ITEM):
    # Giao dịch: list product_id mỗi bill
    g = bills_long.groupby("bill_id")["product_id"].apply(list)
    N_tx = len(g)
    if N_tx == 0:
        cols = ["antecedent_id","consequent_id",
                "antecedent_name","consequent_name",
                "support","confidence","lift","leverage"]
        pd.DataFrame(columns=cols).to_csv(ITEM_RULES_PATH, index=False, encoding="utf-8")
        return pd.DataFrame(columns=cols)

    # Đếm số bill chứa từng sản phẩm
    item_cnt = Counter()
    for items in g:
        for p in set(items):
            item_cnt[p] += 1

    # Đếm cặp sản phẩm cùng bill
    pair_cnt = Counter()
    for items in g:
        uniq = sorted(set(items))
        for i in range(len(uniq)):
            for j in range(i+1, len(uniq)):
                a, b = uniq[i], uniq[j]
                pair_cnt[(a, b)] += 1

    # Map product_id -> product_name (đã chuẩn hoá từ cell cleaning products)
    pid2name = products_df.set_index("product_id")["product_name"].to_dict()

    rows = []
    for (a, b), c_ab in pair_cnt.items():
        sup = c_ab / N_tx
        if sup < min_support:
            continue

        c_a = item_cnt[a]
        c_b = item_cnt[b]
        p_a = c_a / N_tx
        p_b = c_b / N_tx

        # A -> B
        conf_ab = c_ab / c_a if c_a > 0 else 0.0
        lift_ab = conf_ab / p_b if p_b > 0 else 0.0
        lev_ab  = sup - p_a * p_b
        if conf_ab >= min_conf and lift_ab >= min_lift:
            rows.append((
                a, b,
                pid2name.get(a, ""), pid2name.get(b, ""),
                sup, conf_ab, lift_ab, lev_ab
            ))

        # B -> A
        conf_ba = c_ab / c_b if c_b > 0 else 0.0
        lift_ba = conf_ba / p_a if p_a > 0 else 0.0
        if conf_ba >= min_conf and lift_ba >= min_lift:
            rows.append((
                b, a,
                pid2name.get(b, ""), pid2name.get(a, ""),
                sup, conf_ba, lift_ba, lev_ab
            ))

    cols = ["antecedent_id","consequent_id",
            "antecedent_name","consequent_name",
            "support","confidence","lift","leverage"]

    if not rows:
        rules_df = pd.DataFrame(columns=cols)
    else:
        rules_df = pd.DataFrame(rows, columns=cols)
        rules_df.sort_values(["lift","confidence","support"], ascending=False, inplace=True)

    rules_df.to_csv(ITEM_RULES_PATH, index=False, encoding="utf-8")
    return rules_df

item_rules = mine_item_rules_pairwise(
    bills_long,
    products_df=products,
    min_support=MIN_SUPPORT_ITEM,
    min_conf=MIN_CONF_ITEM,
    min_lift=MIN_LIFT_ITEM
)

In [ ]:
# Luật theo NGÀNH HÀNG (category co-purchase) -> category_rules.csv

def mine_category_rules_pairwise(bills_long, products_df,
                                 min_support=MIN_SUPPORT_CATEGORY,
                                 min_conf=MIN_CONF_CATEGORY,
                                 min_lift=MIN_LIFT_CATEGORY):
    pid2cat = products_df.set_index("product_id")["category"].to_dict()

    # Giao dịch theo category
    g_items = bills_long.groupby("bill_id")["product_id"].apply(list)

    cat_tx = []
    for tx in g_items:
        cats = {pid2cat.get(p, "UNCATEGORIZED") for p in tx}
        cats = [c for c in cats if c and c != "UNCATEGORIZED"]
        if len(cats) >= 2:
            cat_tx.append(sorted(set(cats)))

    N_tx = len(cat_tx)
    if N_tx == 0:
        pd.DataFrame(columns=["category_A","category_B","support","confidence","lift","leverage"])\
          .to_csv(CATEGORY_RULES_PATH, index=False, encoding="utf-8")
        return pd.DataFrame()

    # Đếm category đơn
    cat_cnt = Counter()
    for cats in cat_tx:
        for c in set(cats):
            cat_cnt[c] += 1

    # Đếm cặp category
    pair_cnt = Counter()
    for cats in cat_tx:
        for i in range(len(cats)):
            for j in range(i+1, len(cats)):
                a, b = cats[i], cats[j]
                pair_cnt[(a, b)] += 1

    rows = []
    for (a, b), c_ab in pair_cnt.items():
        sup = c_ab / N_tx
        if sup < min_support:
            continue

        c_a = cat_cnt[a]
        c_b = cat_cnt[b]
        p_a = c_a / N_tx
        p_b = c_b / N_tx

        # A -> B
        conf_ab = c_ab / c_a if c_a > 0 else 0.0
        lift_ab = conf_ab / p_b if p_b > 0 else 0.0
        lev_ab  = sup - p_a * p_b
        if conf_ab >= min_conf and lift_ab >= min_lift:
            rows.append((a, b, sup, conf_ab, lift_ab, lev_ab))

        # B -> A
        conf_ba = c_ab / c_b if c_b > 0 else 0.0
        lift_ba = conf_ba / p_a if p_a > 0 else 0.0
        if conf_ba >= min_conf and lift_ba >= min_lift:
            rows.append((b, a, sup, conf_ba, lift_ba, lev_ab))

    if not rows:
        rules_df = pd.DataFrame(columns=["category_A","category_B","support","confidence","lift","leverage"])
    else:
        rules_df = pd.DataFrame(rows, columns=["category_A","category_B","support","confidence","lift","leverage"])
        rules_df.sort_values(["lift","confidence","support"], ascending=False, inplace=True)

    # Lưu file cuối
    rules_df.to_csv(CATEGORY_RULES_PATH, index=False, encoding="utf-8")
    return rules_df

category_rules = mine_category_rules_pairwise(
    bills_long, products,
    min_support=MIN_SUPPORT_CATEGORY,
    min_conf=MIN_CONF_CATEGORY,
    min_lift=MIN_LIFT_CATEGORY
)